In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8MB 3.6MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617564 sha256=a4f8936a378de0145cccb27717f0fa704161228228539ac383a4e1df84b670e0
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import surprise as sp
import seaborn as sns
import scipy

from math import sqrt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.decomposition import TruncatedSVD
from surprise import KNNBasic,accuracy
from surprise.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive"

 books.csv	       BX-Books.csv	'Getting started.pdf'
 book_tags.csv	       BX-CSV-Dump.zip	 ratings.csv
 BX-Book-Ratings.csv   BX-Users.csv	 tags.csv


In [ ]:
books_df = pd.read_csv('/content/drive/MyDrive/books.csv')
ratings_df = pd.read_csv('/content/drive/MyDrive/ratings.csv')
books_df.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [ ]:
book_tags = pd.read_csv('/content/drive/MyDrive/book_tags.csv', encoding = "ISO-8859-1")
book_tags.head()
tags = pd.read_csv('/content/drive/MyDrive/tags.csv')
tags.tail()
tags_join_DF = pd.merge(book_tags, tags, left_on='tag_id', right_on='tag_id', how='inner')
tags_join_DF.head()
books_with_tags = pd.merge(books_df, tags_join_DF, left_on='book_id', right_on='goodreads_book_id', how='inner')

In [ ]:
# removing all columns that we will not use

ignore_columns = ['books_count', 'isbn', 'best_book_id', 'goodreads_book_id', 'work_id', 'isbn13', 'language_code', 'work_ratings_count', 'work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url', 'small_image_url']
books_df = books_df.drop(ignore_columns, axis = 1)
books_df.head()

,book_id,authors,original_publication_year,original_title,title,average_rating,ratings_count
0,1,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",4.34,4780653
1,2,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,4.44,4602479
2,3,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",3.57,3866839
3,4,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,4.25,3198671
4,5,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,3.89,2683664


In [ ]:
# removing books that have less than 50 ratings, also removing books that have 0 ratings

popularity_threshold = 50
books_df = books_df.query('ratings_count >= @popularity_threshold')
rating_threshold = 1.0
books_df = books_df.query('average_rating >= @rating_threshold')
ignore_columns = ['average_rating']
books_df = books_df.drop(ignore_columns, axis = 1)

In [ ]:
combine_rating_df = pd.merge(ratings_df, books_df, on='book_id')
ignore_columns = ['authors', 'original_publication_year', 'original_title', 'title']
combine_rating_df = combine_rating_df.drop(ignore_columns, axis=1)
combine_rating_df.head()

,user_id,book_id,rating,ratings_count
0,1,258,5,263685
1,11,258,3,263685
2,143,258,4,263685
3,242,258,5,263685
4,325,258,4,263685


In [ ]:
# For Content-Based Filtering

temp_df = books_with_tags.groupby('book_id')['tag_name'].apply(' '.join).reset_index()
temp_df.head()
tag_books_df = pd.merge(books_df, temp_df, left_on='book_id', right_on='book_id', how='inner')
tag_books_df['corpus'] = (pd.Series(tag_books_df[['authors', 'tag_name']]
                .fillna('')
                .values.tolist()
                ).str.join(' '))

In [ ]:
# For Content-Based Filtering

def find_recommendation(book_title):
  tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
  tfidf_matrix = tf.fit_transform(books_df['authors'])
  cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
  indices = pd.Series(books_df.index, index=books_df['title'])
  
  tf_corpus = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
  tfidf_matrix_corpus = tf_corpus.fit_transform(tag_books_df['corpus'])
  cosine_sim_corpus = linear_kernel(tfidf_matrix_corpus, tfidf_matrix_corpus)

  # Build a 1-dimensional array with book titles
  titles = tag_books_df['title']
  indices = pd.Series(tag_books_df.index, index=tag_books_df['title'])
  titles1 = books_df['title']
  indices1 = pd.Series(books_df.index, index=books_df['title'])

  # Function that get book recommendations based on the cosine similarity score of books tags
  def corpus_recommendations(title):
      idx = indices1[title]
      sim_scores = list(enumerate(cosine_sim_corpus[idx]))
      sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
      sim_scores = sim_scores[1:21]
      book_indices = [i[0] for i in sim_scores]
      return titles.iloc[book_indices]

  ret = corpus_recommendations(book_title)
  return ret

In [ ]:
def collaborative_filtering(books_input, subsetGroup):
  pearsonCorrelationDict = {}
  subsetGroup = subsetGroup[0: 1000]

  #For every user group in our subset
  for name, group in subsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='book_id')
    books_input = books_input.sort_values(by='book_id')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = books_input[books_input['book_id'].isin(group['book_id'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
      pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
      pearsonCorrelationDict[name] = 0

    pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
    pearsonDF.columns = ['similarityIndex']
    pearsonDF['user_id'] = pearsonDF.index
    pearsonDF.index = range(len(pearsonDF))
    topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
    topUsersRating=topUsers.merge(ratings_df, left_on='user_id', right_on='user_id', how='inner')
    topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
    tempTopUsersRating = topUsersRating.groupby('book_id').sum()[['similarityIndex','weightedRating']]
    tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
    #Creates an empty dataframe
    recommendation_df = pd.DataFrame()
    #Now we take the weighted average
    recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
    recommendation_df['book_id'] = tempTopUsersRating.index
    recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
    return recommendation_df
    


In [ ]:
def do_collaborative_filtering(books_input, subsetGroup):
  recommendation_df = collaborative_filtering(books_input, subsetGroup)
  temp_df = books_df.loc[books_df['book_id'].isin(recommendation_df.head(15)['book_id'].tolist())]
  temp_df[["title", "authors", "original_publication_year"]].rename(columns={'title': 'Title', 'authors': 'Authors','original_publication_year': "Publication"})
  
  return temp_df.head(15)

In [ ]:
def do_content_based_filtering(books_input, userInput):
  for i in range(0, len(userInput)):
    title = inputBooks2['original_title'][i]
    recommendation_df = pd.DataFrame()
    recommendation_df = find_recommendation(title)
    recommendation_df.columns = ['book_id', 'title']
    print("Book Recommendations for '{}': \n".format(title))
    print(recommendation_df.head(5))

In [ ]:
def hybrid_recommendation_system(book_input, userInput, userSub, userSubGroup):
  similar_users = len(userSub)
  if similar_users > 4000:
    recommendation_df = do_collaborative_filtering(book_input, userSubsetGroup)
    return recommendation_df[["title", "authors", "original_publication_year"]].rename(columns={'title': 'Title', 'authors': 'Authors','original_publication_year': "Publication"})
  else:
    do_content_based_filtering(book_input, userInput)

In [ ]:
userInput1 = [
            {'original_title':'The Hound of the Baskervilles', 'rating': 5},
            {'original_title': 'A Study in Scarlet', 'rating': 4},
            {'original_title':'The Da Vinci Code', 'rating': 4},
            {'original_title':'The Murder of Roger Ackroyd', 'rating': 5},
            {'original_title':"The Day of the Jackal", 'rating': 4},
            {'original_title':'Hell House', 'rating': 3}
         ] 
inputBooks1 = pd.DataFrame(userInput1)
inputBooks1

,original_title,rating
0,The Hound of the Baskervilles,5
1,A Study in Scarlet,4
2,The Da Vinci Code,4
3,The Murder of Roger Ackroyd,5
4,The Day of the Jackal,4
5,Hell House,3


In [ ]:
inputId1 = books_df[books_df['original_title'].isin(inputBooks1['original_title'].tolist())]
inputBooks1 = pd.merge(inputId, inputBooks1)
ignore_columns = ['authors', 'original_publication_year', 'original_title', 'title']
inputBooks1 = inputBooks1.drop(ignore_columns, axis = 1)
inputBooks1

,book_id,ratings_count,rating
0,26,1447148,4
1,446,201499,4
2,618,154517,5
3,989,85883,4
4,1410,71354,5
5,4613,22644,3


In [ ]:
userSubset = ratings_df[ratings_df['book_id'].isin(inputBooks1['book_id'].tolist())]
rating_threshold = 1.0
userSubset = userSubset.query('rating >= @rating_threshold')
userSubset.head()

,user_id,book_id,rating
5,2,26,4
27,4,26,3
119,9,26,2
171,11,26,3
197,15,26,3


In [ ]:
userSubsetGroup = userSubset.groupby(['user_id'])
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
userSubsetGroup[0:3]

[(14623,          user_id  book_id  rating
  919755     14623      989       5
  920210     14623       26       4
  1157209    14623      618       3
  1157816    14623     1410       5
  1167503    14623      446       4), (34584,          user_id  book_id  rating
  3013135    34584       26       4
  3022788    34584      989       3
  3023088    34584     1410       3
  3023095    34584      446       3
  3023097    34584      618       4), (35451,          user_id  book_id  rating
  2785573    35451      446       3
  3147839    35451     1410       3
  3147856    35451      618       3
  3147877    35451      989       4
  3147882    35451       26       3)]

In [ ]:
len(userSubset)

19501

In [ ]:
hybrid_recommendation_system(inputBooks1, userInput1, userSubset, userSubsetGroup)

,Title,Authors,Publication
1,Harry Potter and the Sorcerer's Stone (Harry P...,"J.K. Rowling, Mary GrandPré",1997.0
3,To Kill a Mockingbird,Harper Lee,1960.0
26,Harry Potter and the Half-Blood Prince (Harry ...,"J.K. Rowling, Mary GrandPré",2005.0
254,Atlas Shrugged,"Ayn Rand, Leonard Peikoff",1957.0
273,The Godfather,"Mario Puzo, Robert Thompson, Peter Bart",1969.0
286,The Fountainhead,"Ayn Rand, Leonard Peikoff",1943.0
513,The Adventures of Sherlock Holmes,Arthur Conan Doyle,1892.0
988,The Day of the Jackal,Frederick Forsyth,1971.0
1409,"The Murder of Roger Ackroyd (Hercule Poirot, #4)",Agatha Christie,1926.0
2407,"Love Story (Love Story, #1)",Erich Segal,1970.0


In [ ]:
userInput2 = [
            {'original_title': 'The Graveyard Book', 'rating': 5}, 
         ] 
inputBooks2 = pd.DataFrame(userInput2)
inputBooks2

,original_title,rating
0,The Graveyard Book,5


In [ ]:
inputId2 = books_df[books_df['original_title'].isin(inputBooks2['original_title'].tolist())]
inputBooks2 = pd.merge(inputId2, inputBooks2)
ignore_columns = ['authors', 'original_publication_year', 'original_title', 'title']
inputBooks2_tmp = inputBooks2.drop(ignore_columns, axis = 1)
inputBooks2_tmp

,book_id,ratings_count,rating
0,312,302303,5


In [ ]:
userSubset2 = ratings_df[ratings_df['book_id'].isin(inputBooks2_tmp['book_id'].tolist())]
rating_threshold = 1.0
userSubset2 = userSubset2.query('rating >= @rating_threshold')
userSubset2.head()

,user_id,book_id,rating
236311,5858,312,5
290697,5862,312,5
297434,5966,312,4
298344,3980,312,5
301687,3936,312,4


In [ ]:
userSubsetGroup2 = userSubset2.groupby(['user_id'])
userSubsetGroup2 = sorted(userSubsetGroup2,  key=lambda x: len(x[1]), reverse=True)
userSubsetGroup2[0:3]

[(31,         user_id  book_id  rating
  588924       31      312       4), (36,         user_id  book_id  rating
  523411       36      312       5), (50,         user_id  book_id  rating
  355757       50      312       5)]

In [ ]:
len(userSubset2)

3401

In [ ]:
hybrid_recommendation_system(inputBooks2, userInput2, userSubset2, userSubsetGroup2)

Book Recommendations for 'The Graveyard Book': 

713    Two Ravens and One Crow (The Iron Druid Chroni...
292                                   North of Beautiful
107                               Between Shades of Gray
293                                               Willow
400                                   The Sheltering Sky
Name: title, dtype: object
